In [0]:
%scala
System.setProperty("ssl.ca.location","dbfs:/FileStore/tables/cloudkarafka.ca");

res1: String = dbfs:/FileStore/tables/cloudkarafka.ca

In [0]:
#dbutils.fs.rm("dbfs:/FileStore/tables/cloudkarafka.jaas")
dbutils.fs.head("dbfs:/FileStore/tables/cloudkarafka.jaas")

Out[1]: 'KafkaClient {\n org.apache.kafka.common.security.scram.ScramLoginModule required\n username="yrmfqh3q"\n password="WfKJEYL_hwNUHVU8laSbj6gXkPIw_xuc";\n};'

In [0]:
dbutils.fs.rm("/tmp/stream/pykafka_test/", True);

UsageError: Line magic function `%fs` not found.

In [0]:
%fs mkdirs /tmp/stream/pykafka_test/

res3: Boolean = true

In [0]:
%scala
System.clearProperty("java.security.auth.login.config");
System.clearProperty("sasl.jaas.config");
System.setProperty("java.security.auth.login.config", "dbfs:/FileStore/tables/cloudkarafka.jaas");
System.setProperty("sasl.jaas.config", "dbfs:/FileStore/tables/cloudkarafka.jaas");
println("auth.lang:" + System.getProperty("java.security.auth.login.config"));
println("sasl.jaas:" + System.getProperty("sasl.jaas.config"));

//spark.conf.set("spark.driver.extraJavaOptions", "-Djava.security.auth.login.config=/FileStore/tables/cloudkarafka.jaas")
//spark.conf.set("spark.executor.extraJavaOptions", "-Djava.security.auth.login.config=/FileStore/tables/cloudkarafka.jaas")
spark.sparkContext.addFile("dbfs:/FileStore/tables/kafka_clients_2_4_1.jar")
spark.sparkContext.addFile("dbfs:/FileStore/tables/spark_streaming_kafka_0_10_2_12_3_0_1.jar")
spark.sparkContext.addFile("dbfs:/FileStore/tables/cloudkarafka.jaas")

println("driver.extra  :" + spark.conf.get("spark.driver.extraJavaOptions"));
println("executor.extra:" + spark.conf.get("spark.executor.extraJavaOptions"));


auth.lang:dbfs:/FileStore/tables/cloudkarafka.jaas
sasl.jaas:dbfs:/FileStore/tables/cloudkarafka.jaas
driver.extra :-Djava.security.auth.login.config=dbfs:/FileStore/tables/cloudkarafka.jaas
executor.extra:-Djava.io.tmpdir=/local_disk0/tmp -XX:ReservedCodeCacheSize=256m -XX:+UseCodeCacheFlushing -Djava.security.properties=/databricks/spark/dbconf/java/extra.security -XX:-UseContainerSupport -XX:+PrintFlagsFinal -XX:+PrintGCDateStamps -verbose:gc -XX:+PrintGCDetails -Xss4m -Djava.library.path=/usr/java/packages/lib/amd64:/usr/lib64:/lib64:/lib:/usr/lib:/usr/lib/x86_64-linux-gnu/jni:/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu:/usr/lib/jni -Djavax.xml.datatype.DatatypeFactory=com.sun.org.apache.xerces.internal.jaxp.datatype.DatatypeFactoryImpl -Djavax.xml.parsers.DocumentBuilderFactory=com.sun.org.apache.xerces.internal.jaxp.DocumentBuilderFactoryImpl -Djavax.xml.parsers.SAXParserFactory=com.sun.org.apache.xerces.internal.jaxp.SAXParserFactoryImpl -Djavax.xml.validation.SchemaFactory:http://www.w3.org/2001/XMLSchema=com.sun.org.apache.xerces.internal.jaxp.validation.XMLSchemaFactory -Dorg.xml.sax.driver=com.sun.org.apache.xerces.internal.parsers.SAXParser -Dorg.w3c.dom.DOMImplementationSourceList=com.sun.org.apache.xerces.internal.dom.DOMXSImplementationSourceImpl -Djavax.net.ssl.sessionCacheSize=10000 -Dscala.reflect.runtime.disable.typetag.cache=true -Djava.security.auth.login.config=dbfs:/FileStore/tables/cloudkarafka.jaas -Ddatabricks.serviceName=spark-executor-1

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("Spark-Karafka-Test") \
    .getOrCreate()
#https://docs.confluent.io/3.0.0/kafka/sasl.html
#kafkashaded.org.apache.kafka.common.errors.UnsupportedSaslMechanismException: Client SASL mechanism 'PLAIN' not enabled in the server, enabled mechanisms are [SCRAM-SHA-256]
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "omnibus-01.srvs.cloudkafka.com:9094,omnibus-02.srvs.cloudkafka.com:9094,omnibus-03.srvs.cloudkafka.com:9094") \
    .option("kafka.security.protocol", "SASL_SSL") \
    .option("kafka.sasl.mechanism", "SCRAM-SHA-256") \
    .option("security.protocol", "SASL_SSL") \
    .option("sasl.mechanism", "SCRAM-SHA-256") \
    .option("subscribe", "yrmfqh3q-test") \
    .load()
print("df1:", df)

df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
print("df2:", df)

ds = df \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "omnibus-01.srvs.cloudkafka.com:9094,omnibus-02.srvs.cloudkafka.com:9094,omnibus-03.srvs.cloudkafka.com:9094") \
  .option("kafka.security.protocol", "SASL_SSL") \
  .option("kafka.sasl.mechanism", "SCRAM-SHA-256") \
  .option("security.protocol", "SASL_SSL") \
  .option("sasl.mechanism", "SCRAM-SHA-256") \
  .option("checkpointLocation", "/tmp/stream/pykafka_test/") \
  .option("topic", "yrmfqh3q-default") \
  .start()
print("ds:", ds)


df1: DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]
df2: DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]
ds: <pyspark.sql.streaming.StreamingQuery object at 0x7f1bb51f1c50>